In [1]:
# Import necessary libraries
import os
import vtk
import scipy
import slicer
import numpy as np
import pandas as pd
from skimage import measure 
import vtk.util.numpy_support as vtk_np

# Define input directories and filenames
input_folder = ""  # Path to the folder containing input images
matrix_folder = ""  # Path to the folder containing registration matrices
lesion_output_folder = ""  # Path to the folder for saving lesion results
file = "lesion_test.tif"  # The filename of the TIFF image to process
matrix_filename = "lesion_registration_matrix.h5"  # The filename of the registration matrix

In [2]:
def transformPoint(matrix, point):
    """
    Transforms a 3D point using a given transformation matrix.

    Args:
        matrix (vtk.vtkMatrix4x4): The 4x4 transformation matrix to be applied.
        point (list or array): A 3D point represented as a list or array [x, y, z].

    Returns:
        list: Transformed 3D point as [x', y', z'].
    """

    # Initialize transformed point in homogeneous coordinates
    transformedPoint = [0, 0, 0, 1]
    
    # Perform the matrix multiplication, adding a fourth coordinate for homogeneous transformation
    matrix.MultiplyPoint(point + [1], transformedPoint)
    
    return [transformedPoint[0], transformedPoint[1], transformedPoint[2]]

In [ ]:
# Print a message indicating the start of processing
print("Processing...", file)

In [ ]:
## Loading the image volume
input_filename = file
input_path = os.path.join(input_folder, input_filename)  # Construct the full input file path

# Set options for loading the volume (e.g., hide the view and discard image orientation)
loadOptions = {'show': False, 'discardOrientation': True}

# Load the volume into Slicer and return the associated node
input_volume = slicer.util.loadVolume(input_path, properties=loadOptions, returnNode=True)
input_volume_node = input_volume[1]  # Access the volume node

In [ ]:
## Loading the transformation matrix
matrix_path = os.path.join(matrix_folder, matrix_filename)  # Construct the full matrix file path

# Load the transformation matrix from the specified path
slicer.util.loadTransform(matrix_path)

# Get the transformation node from the Slicer scene, using the matrix filename as a reference
matrix_transform_node = slicer.util.getNode(matrix_filename.replace('.h5', ''))

In [4]:
## Define ROI
bounds = [0]*6
input_volume_node.GetRASBounds(bounds)  # Get the RAS bounds of the input volume

# Define the 8 corners of the bounding box based on the bounds
corners = [
    [bounds[0], bounds[2], bounds[4]],
    [bounds[0], bounds[2], bounds[5]],
    [bounds[0], bounds[3], bounds[4]],
    [bounds[0], bounds[3], bounds[5]],
    [bounds[1], bounds[2], bounds[4]],
    [bounds[1], bounds[2], bounds[5]],
    [bounds[1], bounds[3], bounds[4]],
    [bounds[1], bounds[3], bounds[5]],
]

# Create a matrix for the transformation and get the transformation matrix from the node
transformMatrix = vtk.vtkMatrix4x4()
matrix_transform_node.GetMatrixTransformToParent(transformMatrix)

# Transform each corner using the transformation matrix
transformedCorners = [transformPoint(transformMatrix, corner) for corner in corners]

# Calculate the new bounds for the transformed volume
transformedBounds = [float('inf'), -float('inf')] * 3
for point in transformedCorners:
    for i in range(3):
        transformedBounds[i*2] = min(transformedBounds[i*2], point[i])
        transformedBounds[i*2 + 1] = max(transformedBounds[i*2 + 1], point[i])

# Combine the original bounds with the transformed bounds to determine the new ROI
combinedBounds = [0]*6
for i in range(3):
    combinedBounds[i*2] = min(bounds[i*2], transformedBounds[i*2])
    combinedBounds[i*2 + 1] = max(bounds[i*2 + 1], transformedBounds[i*2 + 1])

# Calculate the new center and radius for the ROI
newCenter = [(combinedBounds[0] + combinedBounds[1]) / 2, (combinedBounds[2] + combinedBounds[3]) / 2, (combinedBounds[4] + combinedBounds[5]) / 2]
newRadius = [(combinedBounds[1] - combinedBounds[0]) / 2, (combinedBounds[3] - combinedBounds[2]) / 2, (combinedBounds[5] - combinedBounds[4]) / 2]

# Create a new ROI node and set its center and radius
roiNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsROINode")
roiNode.SetXYZ(newCenter)
roiNode.SetRadiusXYZ(newRadius)

# Create a new output volume node
outputVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode")

# Crop the volume using the newly defined ROI
cropVolumeLogic = slicer.modules.cropvolume.logic()
cropVolumeParameterNode = slicer.vtkMRMLCropVolumeParametersNode()
cropVolumeParameterNode.SetROINodeID(roiNode.GetID())
cropVolumeParameterNode.SetInputVolumeNodeID(input_volume_node.GetID())
cropVolumeParameterNode.SetVoxelBased(True)
cropVolumeLogic.Apply(cropVolumeParameterNode)
croppedVolume = slicer.mrmlScene.GetNodeByID(cropVolumeParameterNode.GetOutputVolumeNodeID())

## Resample Brain
output_volume_node = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode")

# Set parameters for resampling the cropped volume
resampleModule = slicer.modules.resamplescalarvectordwivolume
parameters = {
    "inputVolume": croppedVolume,
    "outputVolume": output_volume_node,
    "referenceVolume": croppedVolume,
    "transformationFile": matrix_transform_node,
    "interpolationType": "linear"
}

# Run the resampling operation
cliNode = slicer.cli.runSync(resampleModule, None, parameters)

## Redefine ROI based on the resampled volume
newBounds = [float('inf'), -float('inf')] * 3
for point in transformedCorners:
    for i in range(3):
        newBounds[i*2] = min(newBounds[i*2], point[i])
        newBounds[i*2 + 1] = max(newBounds[i*2 + 1], point[i])

# Calculate the new center and radius for the updated ROI
newCenter = [(newBounds[0] + newBounds[1]) / 2, (newBounds[2] + newBounds[3]) / 2, (newBounds[4] + newBounds[5]) / 2]
newRadius = [(newBounds[1] - newBounds[0]) / 2, (newBounds[3] - newBounds[2]) / 2, (newBounds[5] - newBounds[4]) / 2]

# Create a new ROI node and set its center and radius
roiNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsROINode")
roiNode.SetXYZ(newCenter)
roiNode.SetRadiusXYZ(newRadius)

# Create a new output volume node
outputVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode")

# Crop the volume using the updated ROI
cropVolumeLogic = slicer.modules.cropvolume.logic()
cropVolumeParameterNode = slicer.vtkMRMLCropVolumeParametersNode()
cropVolumeParameterNode.SetROINodeID(roiNode.GetID())
cropVolumeParameterNode.SetInputVolumeNodeID(output_volume_node.GetID())
cropVolumeParameterNode.SetVoxelBased(True)
cropVolumeLogic.Apply(cropVolumeParameterNode)
croppedVolume2 = slicer.mrmlScene.GetNodeByID(cropVolumeParameterNode.GetOutputVolumeNodeID())

In [5]:
## Calculate Center of Mass (COM) of the cropped volume

# Get the image data from the cropped volume
imageData = croppedVolume2.GetImageData()

# Get the dimensions of the image data
dims = imageData.GetDimensions()

# Convert VTK array to NumPy array
vtkArray = imageData.GetPointData().GetScalars()
numpyArray = vtk_np.vtk_to_numpy(vtkArray)
numpyArray = numpyArray.reshape(dims[2], dims[1], dims[0])  # Reshape according to the dimensions
print("Array shape:", numpyArray.shape)  # Print the shape for debugging

# Apply a threshold to the array (set values <= 120 to 0)
numpyArray[numpyArray <= 120] = 0

# Calculate the center of mass (COM) of the thresholded array
com = scipy.ndimage.center_of_mass(numpyArray)

# Retrieve the IJK to RAS matrix from the volume node
volumeNode = croppedVolume2
ijkToRasMatrix = vtk.vtkMatrix4x4()
volumeNode.GetIJKToRASMatrix(ijkToRasMatrix)

# Extract the offset values from the matrix
x_offset = ijkToRasMatrix.GetElement(0, 3)
y_offset = ijkToRasMatrix.GetElement(1, 3)
z_offset = ijkToRasMatrix.GetElement(2, 3)

# Adjust the COM coordinates by adding the offsets
adjusted_com = [com[0] * ijkToRasMatrix.GetElement(0, 0) + x_offset,
                com[1] * ijkToRasMatrix.GetElement(1, 1) + y_offset,
                com[2] * ijkToRasMatrix.GetElement(2, 2) + z_offset]

# Print the adjusted center of mass coordinates
print("Center of mass coordinate:", adjusted_com)

In [7]:
## Create a new fiducial markups node in the scene
markupsNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLMarkupsFiducialNode')

# Add a fiducial at the center of mass coordinates, adjusted by the offsets
markupsNode.AddFiducial(com[2] + y_offset, com[1] + x_offset, com[0] + z_offset)

# Set the name of the markups node to "CenterOfMass"
markupsNode.SetName("CenterOfMass")

In [9]:
## Create a new plane markups node in the scene
planeNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLMarkupsPlaneNode')

# Set the origin of the plane to the center of mass coordinates, adjusted by the offsets
planeNode.SetOrigin(com[2] + y_offset, com[1], com[0] + z_offset)

# Define the width and height of the plane
planeWidth = 300
planeHeight = 300

# Set the size of the plane
planeNode.SetSize(planeWidth, planeHeight)

# Set the normal vector of the plane (orienting it along the z-axis)
planeNode.SetNormal(0, 0, 1)

# Name the plane node for easy identification
planeNode.SetName("COM_Plane")

In [11]:
## Split Brain

# Determine the slice index based on the center of mass coordinate
slice_index = int(com[0])

# Create a copy of the original array for slicing
slicedArray = numpyArray.copy()

# Zero out the voxels in one half of the array (up to the slice_index)
slicedArray[0:slice_index, :, :] = 0

# Create a new volume node for the right side of the brain
rightNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLScalarVolumeNode')
slicer.util.updateVolumeFromArray(rightNode, slicedArray)
rightNode.SetName("Right")  # Set the name of the right volume node

# Recreate the slicedArray for the left side of the brain
slicedArray = numpyArray.copy()

# Zero out the voxels in the other half of the array (from slice_index onward)
slicedArray[slice_index:, :, :] = 0

# Create a new volume node for the left side of the brain
leftNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLScalarVolumeNode')
slicer.util.updateVolumeFromArray(leftNode, slicedArray)
leftNode.SetName("Left")  # Set the name of the left volume node

In [13]:
## Flip left brain

# Create a copy of the original array for the left side of the brain
leftArray = numpyArray.copy()

# Zero out the voxels in the array from the slice_index onward
leftArray[slice_index:, :, :] = 0

# Flip the left side of the brain along the x-axis (axis=0)
flippedLeftArray = np.flip(leftArray, axis=0)

# Create a new volume node for the flipped left side of the brain
flippedLeftNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLScalarVolumeNode')

# Update the new volume node with the flipped array
slicer.util.updateVolumeFromArray(flippedLeftNode, flippedLeftArray)

# Set the name of the flipped left volume node
flippedLeftNode.SetName("Flipped Left")

In [14]:
# Register flipped left brain to right brain

# Select the BRAINSFit module in Slicer
slicer.util.selectModule('BRAINSFit')
brainsFit = slicer.modules.brainsfit

# Get the subject hierarchy node from the scene
shNode = slicer.vtkMRMLSubjectHierarchyNode.GetSubjectHierarchyNode(slicer.mrmlScene)

# Define parameters for rigid registration
parametersRigid = {}
parametersRigid["fixedVolume"] = rightNode.GetID()  # The fixed (reference) volume
parametersRigid["movingVolume"] = flippedLeftNode.GetID()  # The moving (to be aligned) volume

# Create and add a new linear transform node for the registration
linearTransform = slicer.vtkMRMLLinearTransformNode()
linearTransform.SetName('Rigid')
slicer.mrmlScene.AddNode(linearTransform)

# Set additional parameters for the rigid registration
parametersRigid["linearTransform"] = linearTransform.GetID()
parametersRigid["useRigid"] = True  # Specify that we are using rigid registration
parametersRigid["samplingPercentage"] = 0.05  # Sampling percentage for the registration

# Create and add a new scalar volume node for the registered volume
regVolume = slicer.vtkMRMLScalarVolumeNode()
regVolume.SetName("Flipped Left Reg")
slicer.mrmlScene.AddNode(regVolume)
parametersRigid["outputVolume"] = regVolume.GetID()

# Set registration parameters
parametersRigid["maximumStepLength"] = 0.05  # Maximum step length for the optimization
parametersRigid["minimumStepLength"] = 0.001  # Minimum step length for the optimization
parametersRigid["maxIterations"] = 3000  # Maximum number of iterations

# Run the BRAINSFit module with the defined parameters
cliBrainsFitRigidNode = None
cliBrainsFitRigidNode = slicer.cli.runSync(brainsFit, None, parametersRigid)

In [15]:
## Segment!

# Create a new segmentation node and add it to the scene
segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
segmentationNode.CreateDefaultDisplayNodes()  # Create default display nodes for visualization

# Set the reference image geometry based on the volume node
segmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(rightNode)

# Add an empty segment named "lesion" to the segmentation node
addedSegmentID = segmentationNode.GetSegmentation().AddEmptySegment("lesion")

# Create and configure the Segment Editor widget
segmentEditorWidget = slicer.qMRMLSegmentEditorWidget()
segmentEditorWidget.setMRMLScene(slicer.mrmlScene)
segmentEditorNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentEditorNode")
segmentEditorWidget.setMRMLSegmentEditorNode(segmentEditorNode)
segmentEditorWidget.setSegmentationNode(segmentationNode)
segmentEditorWidget.setMasterVolumeNode(rightNode)

# Apply the "Threshold" effect to segment the image
segmentEditorWidget.setActiveEffectByName("Threshold")
effect = segmentEditorWidget.activeEffect()

# Convert volume to numpy array and determine the maximum intensity
arrayData = slicer.util.arrayFromVolume(rightNode)
maxIntensity = np.max(arrayData)

# Set threshold parameters for segmentation
effect.setParameter("MinimumThreshold", "120")
effect.setParameter("MaximumThreshold", str(maxIntensity))
effect.self().onApply()  # Apply the effect

# Apply the "Mask volume" effect to the segmentation
segmentEditorWidget.setActiveEffectByName("Mask volume")
effect = segmentEditorWidget.activeEffect()

# Set the input volume for the mask volume effect
effect.setParameter("Operation", "FILL_INSIDE")
effect.parameterSetNode().SetNodeReferenceID("Mask volume.InputVolume", regVolume.GetID())
effect.self().onApply()  # Apply the effect
outputVolumeID = effect.parameterSetNode().GetNodeReferenceID("Mask volume.OutputVolume")
outputVolume = slicer.mrmlScene.GetNodeByID(outputVolumeID)

# Apply the "Mask volume" effect again (this may be redundant)
segmentEditorWidget.setActiveEffectByName("Mask volume")
effect = segmentEditorWidget.activeEffect()
effect.parameterSetNode().SetNodeReferenceID("Mask volume.InputVolume", regVolume.GetID())
effect.self().onApply()  # Apply the effect again
outputVolumeID = effect.parameterSetNode().GetNodeReferenceID("Mask volume.OutputVolume")
outputVolume = slicer.mrmlScene.GetNodeByID(outputVolumeID)

In [ ]:
## Convert to Binary

# Convert the output volume to a binary image where values >= 120 are set to 1, else 0
arrayData = slicer.util.arrayFromVolume(outputVolume)
originalDims = outputVolume.GetImageData().GetDimensions()
binaryImage = np.where(arrayData >= 120, 1, 0)

# Create a new volume node to store the binary image
binaryImageNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLScalarVolumeNode')
slicer.util.updateVolumeFromArray(binaryImageNode, binaryImage)
binaryImageNode.SetName("Binary")

## Find Labels
# Perform connected-component labeling on the binary image
labels = measure.label(binaryImage, connectivity=1)
properties = measure.regionprops(labels)

# Sort the labeled regions based on their area in descending order
sorted_properties = sorted(properties, key=lambda p: p.area, reverse=True)

# Transpose the binary image for later processing
binaryImage_T = binaryImage.transpose(2, 1, 0)

In [ ]:
## Locate lesion
# Process the top five largest regions
top_fives = sorted_properties[:5]
for i, prop in enumerate(top_fives):
    # Create a mask for the current label
    region_mask = labels == prop.label
    region_image = binaryImage * region_mask
    print(f'Volume {i}: ', prop.area)

    # Transpose the mask and region image
    region_mask_T = region_mask.transpose(2, 1, 0)
    region_image_T = binaryImage_T * region_mask_T

    # Optionally, save the node as NRRD (currently commented out)
    # output_filename = file.replace('561_32xds.tif', f'lesion_{i+1}.nrrd')
    # output_path = lesion_output_folder + output_filename  # Replace with your desired file path
    # nrrd.write(output_path, region_image_T)

    # Create a new volume node for each labeled region
    region_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLScalarVolumeNode')
    slicer.util.updateVolumeFromArray(region_node, region_image)
    region_node.SetName(f"Volume_{i+1}")

print('Completed!')

In [ ]:
# Optionally clear the MRML scene (currently commented out)
# slicer.mrmlScene.Clear()